In [5]:
import pandas as pd
import numpy as np
import geopandas as gpd
import transbigdata as tbd
import warnings
warnings.filterwarnings('ignore')
import cplex
from cplex.exceptions import CplexError
from utils import FlowesetimationV2
input_networks = '中间数据/network/'
input_abm = '中间数据/abminput/'
OD_count_path = '中间数据/ODpredict/OD_count/'

ModuleNotFoundError: No module named 'utils'

In [3]:
speed = 100000
#每小时100km
od_dis_table = pd.read_csv(input_networks+r'od_dis_table.csv')
edge_to_charge = pd.read_csv(input_networks+r'edge_to_charge.csv')

100000.0

In [ ]:
# 读入订单数据

edge_order_hour = pd.read_csv(input_abm+'edge_order_hour.csv')

In [ ]:
charge_edge_ids = edge_to_charge['edge_id'].unique()

#筛选出od_dis_table中edge_id在charge_edge_ids中的OD对
od_dis_table = od_dis_table[od_dis_table['edge_id'].isin(charge_edge_ids)]
od_dis_table['cost_time_step'] = np.floor(od_dis_table['cumsumlength']/speed)

od_prob = pd.read_csv(+'/OD_flow_20230116.csv').rename(columns={'flow': 'count'})


#按照count的值排序，筛选出前10%的OD对
od_prob = od_prob.sort_values('count', ascending=False)
od_fix = od_prob.iloc[:int(len(od_prob)*0.1)]

In [6]:
model = cplex.Cplex()

# 设置问题类型为最小化
model.objective.set_sense(model.objective.sense.minimize)

# 添加变量
# 为每个OD对和路段的组合创建24个小时的变量
var_names = [f"x_{row['OD_pairs']}_{row['edge_id']}_{hour}" for _, row in od_fix.iterrows() for hour in range(24)]
model.variables.add(names=var_names, lb=[0]*len(var_names))

# 定义目标函数（这里仅为简化的例子）
model.objective.set_linear([(name, 1.0) for name in var_names])

# 添加车流量约束：确保每个路段在每个小时的车流量不超过上限
for _, edge_row in edge_order_hour.iterrows():
    segment = edge_row['edge_id']
    hour = edge_row['hour']
    flow_limit = edge_row['real_count']
    
    involved_vars = [
        name for name in var_names if f"_{segment}_{hour}" in name
    ]
    coeffs = [1.0] * len(involved_vars)
    model.linear_constraints.add(
        lin_expr=[cplex.SparsePair(ind=involved_vars, val=coeffs)],
        senses=["E"],
        rhs=[flow_limit]
    )
    
# 求解模型
model.solve()

# 打印结果
print("Solution status =", model.solution.get_status(), ":",)
print(model.solution.status[model.solution.get_status()])
print("Solution value  =", model.solution.get_objective_value())

# 打印每个决策变量的解
numcols = model.variables.get_num()
for j in range(numcols):
    print("%s: %10f" % (model.variables.get_names(j), model.solution.get_values(j)))

# 提取决策变量及其解值
solution_values = model.solution.get_values()

# 决策变量的名称和对应的解值
var_names = model.variables.get_names()
var_values = {var_names[i]: solution_values[i] for i in range(len(var_names))}

# 解析决策变量名称，并创建一个包含OD对、路段、小时和流量的DataFrame
data_for_merge = []

for var_name, value in var_values.items():
    # 假设变量名格式为"x_OD_segment_hour"
    parts = var_name.split('_')
    od, edge_id, hour = parts[1], parts[2], parts[3]
    data_for_merge.append({'OD': od, 'edge_id': int(edge_id), 'hour': int(hour), 'flow': value})

# 创建新DataFrame
solution_df = pd.DataFrame(data_for_merge)


NameError: name 'cplex' is not defined